In [1]:
using CSV, DataFrames, JuMP, Gurobi, Plots, HDF5, JLD

In [13]:
# Some constants for the model
alpha = 0.1 # Hospitalization rate for non-vaccinated
beta = 0.02 ;# Hospitalization rate for single vaccinated 
beta_2 = 0.005 ; # Hospitalization rate for fully vaccinated 
gamma = 0.1 # State can only vaccinate 10% of population per week
wait_period = 5 # Number of weeks to wait before taking 2nd dose
file_name = "solution_real" ; # JLD file to save the solution variables in for later analysis 

In [3]:
data_folder = "real_world_data"
# Load csv files
case_file = data_folder * "/" * "case_data.csv"
population_file = data_folder * "/" * "population_data.csv"
vaccine_file = data_folder * "/" * "vaccine_data.csv"

case_data = CSV.read(case_file, DataFrame)
population_data = CSV.read(population_file, DataFrame)
vaccine_data = CSV.read(vaccine_file, DataFrame)


P = Matrix(population_data)
C = Matrix(case_data)
V = Matrix(vaccine_data)
# convert population data to matrix

C =C'
V = V[:,1]
P = P[1,:]

println("C: ", size(C))
println("P: ", size(P))
println("V: ", size(V))

C: (50, 52)
P: (50,)
V: (52,)


In [6]:
n_states = 1:size(case_data, 2)
n_weeks = 1:size(case_data, 1) 
n_weeks_2 = 1:size(case_data, 1)-1 
n_doses = 1:2 

total_states = length(n_states) 
total_weeks = length(n_weeks) 
println("Number of states: ", n_states)
println("Number of weeks: ", n_weeks)

Number of states: 1:50
Number of weeks: 1:52


In [5]:
model = Model(Gurobi.Optimizer)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-24


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

In [7]:
@variable(model, X[n_states, n_weeks, n_doses] >= 0);
X

3-dimensional DenseAxisArray{VariableRef,3,...} with index sets:
    Dimension 1, 1:50
    Dimension 2, 1:52
    Dimension 3, 1:2
And data, a 50×52×2 Array{VariableRef, 3}:
[:, :, 1] =
 X[1,1,1]   X[1,2,1]   X[1,3,1]   …  X[1,50,1]   X[1,51,1]   X[1,52,1]
 X[2,1,1]   X[2,2,1]   X[2,3,1]      X[2,50,1]   X[2,51,1]   X[2,52,1]
 X[3,1,1]   X[3,2,1]   X[3,3,1]      X[3,50,1]   X[3,51,1]   X[3,52,1]
 X[4,1,1]   X[4,2,1]   X[4,3,1]      X[4,50,1]   X[4,51,1]   X[4,52,1]
 X[5,1,1]   X[5,2,1]   X[5,3,1]      X[5,50,1]   X[5,51,1]   X[5,52,1]
 X[6,1,1]   X[6,2,1]   X[6,3,1]   …  X[6,50,1]   X[6,51,1]   X[6,52,1]
 X[7,1,1]   X[7,2,1]   X[7,3,1]      X[7,50,1]   X[7,51,1]   X[7,52,1]
 X[8,1,1]   X[8,2,1]   X[8,3,1]      X[8,50,1]   X[8,51,1]   X[8,52,1]
 X[9,1,1]   X[9,2,1]   X[9,3,1]      X[9,50,1]   X[9,51,1]   X[9,52,1]
 X[10,1,1]  X[10,2,1]  X[10,3,1]     X[10,50,1]  X[10,51,1]  X[10,52,1]
 X[11,1,1]  X[11,2,1]  X[11,3,1]  …  X[11,50,1]  X[11,51,1]  X[11,52,1]
 X[12,1,1]  X[12,2,1]  X[12,3,1]

In [8]:
@variable(model, Y[n_states, n_weeks] >= 0);
Y

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, 1:50
    Dimension 2, 1:52
And data, a 50×52 Matrix{VariableRef}:
 Y[1,1]   Y[1,2]   Y[1,3]   Y[1,4]   Y[1,5]   …  Y[1,50]   Y[1,51]   Y[1,52]
 Y[2,1]   Y[2,2]   Y[2,3]   Y[2,4]   Y[2,5]      Y[2,50]   Y[2,51]   Y[2,52]
 Y[3,1]   Y[3,2]   Y[3,3]   Y[3,4]   Y[3,5]      Y[3,50]   Y[3,51]   Y[3,52]
 Y[4,1]   Y[4,2]   Y[4,3]   Y[4,4]   Y[4,5]      Y[4,50]   Y[4,51]   Y[4,52]
 Y[5,1]   Y[5,2]   Y[5,3]   Y[5,4]   Y[5,5]      Y[5,50]   Y[5,51]   Y[5,52]
 Y[6,1]   Y[6,2]   Y[6,3]   Y[6,4]   Y[6,5]   …  Y[6,50]   Y[6,51]   Y[6,52]
 Y[7,1]   Y[7,2]   Y[7,3]   Y[7,4]   Y[7,5]      Y[7,50]   Y[7,51]   Y[7,52]
 Y[8,1]   Y[8,2]   Y[8,3]   Y[8,4]   Y[8,5]      Y[8,50]   Y[8,51]   Y[8,52]
 Y[9,1]   Y[9,2]   Y[9,3]   Y[9,4]   Y[9,5]      Y[9,50]   Y[9,51]   Y[9,52]
 Y[10,1]  Y[10,2]  Y[10,3]  Y[10,4]  Y[10,5]     Y[10,50]  Y[10,51]  Y[10,52]
 Y[11,1]  Y[11,2]  Y[11,3]  Y[11,4]  Y[11,5]  …  Y[11,50]  Y[11,51]  Y[11,52]
 Y[

In [9]:
@variable(model, W[n_states, n_weeks,n_doses] >= 0);
W

3-dimensional DenseAxisArray{VariableRef,3,...} with index sets:
    Dimension 1, 1:50
    Dimension 2, 1:52
    Dimension 3, 1:2
And data, a 50×52×2 Array{VariableRef, 3}:
[:, :, 1] =
 W[1,1,1]   W[1,2,1]   W[1,3,1]   …  W[1,50,1]   W[1,51,1]   W[1,52,1]
 W[2,1,1]   W[2,2,1]   W[2,3,1]      W[2,50,1]   W[2,51,1]   W[2,52,1]
 W[3,1,1]   W[3,2,1]   W[3,3,1]      W[3,50,1]   W[3,51,1]   W[3,52,1]
 W[4,1,1]   W[4,2,1]   W[4,3,1]      W[4,50,1]   W[4,51,1]   W[4,52,1]
 W[5,1,1]   W[5,2,1]   W[5,3,1]      W[5,50,1]   W[5,51,1]   W[5,52,1]
 W[6,1,1]   W[6,2,1]   W[6,3,1]   …  W[6,50,1]   W[6,51,1]   W[6,52,1]
 W[7,1,1]   W[7,2,1]   W[7,3,1]      W[7,50,1]   W[7,51,1]   W[7,52,1]
 W[8,1,1]   W[8,2,1]   W[8,3,1]      W[8,50,1]   W[8,51,1]   W[8,52,1]
 W[9,1,1]   W[9,2,1]   W[9,3,1]      W[9,50,1]   W[9,51,1]   W[9,52,1]
 W[10,1,1]  W[10,2,1]  W[10,3,1]     W[10,50,1]  W[10,51,1]  W[10,52,1]
 W[11,1,1]  W[11,2,1]  W[11,3,1]  …  W[11,50,1]  W[11,51,1]  W[11,52,1]
 W[12,1,1]  W[12,2,1]  W[12,3,1]

In [10]:
@objective(model, Min, sum(sum(Y[i,j] for i in n_states) for j in n_weeks)) ; 

In [12]:
@constraint(model, hospitalizations[i in n_states,j in n_weeks], Y[i,j] == alpha * C[i,j] * (1 - W[i,j,1]) + beta * C[i,j] * (W[i,j,1] - W[i,j,2]) + beta_2*C[i,j]*W[i,j,2]  ) 

2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, 1:50
    Dimension 2, 1:52
And data, a 50×52 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 hospitalizations[1,1] : Y[1,1] + 2314.32 W[1,1,1] + 433.93500000000006 W[1,1,2] = 2892.9                             …  hospitalizations[1,52] : Y[1,52] + 382.88 W[1,52,1] + 71.78999999999999 W[1,52,2] = 478.6
 hospitalizations[2,1] : Y[2,1] + 177.84 W[2,1,1] + 33.345 W[2,1,2] = 222.3                                              hospitalizations[2,52] : Y[2,52] + 97.52000000000001 W[2,52,1] + 18.285 W[2,52,2] = 121.9
 hospitalizations[3,1] : Y[3,1] + 3519.04 W[3,1,1] + 659.8199999999999 W[3,1,2] = 4398.8                                 hospitalizations[3,52] : Y[3,52

In [14]:
@constraint(model, vaccinated[i in n_states,j in n_weeks_2,k in n_doses], W[i,j+1,k] == W[i,j,k] + X[i,j,k]/ P[i] ) 

3-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},3,...} with index sets:
    Dimension 1, 1:50
    Dimension 2, 1:51
    Dimension 3, 1:2
And data, a 50×51×2 Array{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}, 3}:
[:, :, 1] =
 vaccinated[1,1,1] : -2.0394906576031702e-7 X[1,1,1] - W[1,1,1] + W[1,2,1] = 0      …  vaccinated[1,51,1] : -2.0394906576031702e-7 X[1,51,1] - W[1,51,1] + W[1,52,1] = 0
 vaccinated[2,1,1] : -1.366969906157516e-6 X[2,1,1] - W[2,1,1] + W[2,2,1] = 0          vaccinated[2,51,1] : -1.366969906157516e-6 X[2,51,1] - W[2,51,1] + W[2,52,1] = 0
 vaccinated[3,1,1] : -1.373868499077516e-7 X[3,1,1] - W[3,1,1] + W[3,2,1] = 0          vaccinated[3,51,1] : -1.373868499077516e-7 X[3,51,1] - W[3,51,1] + W[3,52,1] = 0
 vaccinated[4,1,1] : -3.31366781

In [15]:
# Initial values of W are  0 
wait_weeks = 1: wait_period
@constraint(model, initial[i in n_states], W[i,1,1] == 0) 
@constraint(model,initial_2[i in n_states, j in wait_weeks], W[i,j,2]==0)

2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, 1:50
    Dimension 2, 1:5
And data, a 50×5 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 initial_2[1,1] : W[1,1,2] = 0    …  initial_2[1,5] : W[1,5,2] = 0
 initial_2[2,1] : W[2,1,2] = 0       initial_2[2,5] : W[2,5,2] = 0
 initial_2[3,1] : W[3,1,2] = 0       initial_2[3,5] : W[3,5,2] = 0
 initial_2[4,1] : W[4,1,2] = 0       initial_2[4,5] : W[4,5,2] = 0
 initial_2[5,1] : W[5,1,2] = 0       initial_2[5,5] : W[5,5,2] = 0
 initial_2[6,1] : W[6,1,2] = 0    …  initial_2[6,5] : W[6,5,2] = 0
 initial_2[7,1] : W[7,1,2] = 0       initial_2[7,5] : W[7,5,2] = 0
 initial_2[8,1] : W[8,1,2] = 0       initial_2[8,5] : W[8,5,2] = 0
 initial_2[9,1] : W[9,1,2] = 0       ini

In [16]:
# W should be between 0 and 1
@constraint(model, bounds[i in n_states,j in n_weeks,k in n_doses], W[i,j,k] <= 1)

3-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},3,...} with index sets:
    Dimension 1, 1:50
    Dimension 2, 1:52
    Dimension 3, 1:2
And data, a 50×52×2 Array{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}, 3}:
[:, :, 1] =
 bounds[1,1,1] : W[1,1,1] ≤ 1    …  bounds[1,52,1] : W[1,52,1] ≤ 1
 bounds[2,1,1] : W[2,1,1] ≤ 1       bounds[2,52,1] : W[2,52,1] ≤ 1
 bounds[3,1,1] : W[3,1,1] ≤ 1       bounds[3,52,1] : W[3,52,1] ≤ 1
 bounds[4,1,1] : W[4,1,1] ≤ 1       bounds[4,52,1] : W[4,52,1] ≤ 1
 bounds[5,1,1] : W[5,1,1] ≤ 1       bounds[5,52,1] : W[5,52,1] ≤ 1
 bounds[6,1,1] : W[6,1,1] ≤ 1    …  bounds[6,52,1] : W[6,52,1] ≤ 1
 bounds[7,1,1] : W[7,1,1] ≤ 1       bounds[7,52,1] : W[7,52,1] ≤ 1
 bounds[8,1,1] : W[8,1,1] ≤ 1       bounds[8,52,1] : W[8,52,1] ≤ 1

In [17]:
@constraint(model,vaccine_supply[j in n_weeks], sum(sum(X[i,j,k] for k in n_doses) for i in n_states) <= V[j]) 

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:52
And data, a 52-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 vaccine_supply[1] : X[1,1,1] + X[2,1,1] + X[3,1,1] + X[4,1,1] + X[5,1,1] + X[6,1,1] + X[7,1,1] + X[8,1,1] + X[9,1,1] + X[10,1,1] + X[11,1,1] + X[12,1,1] + X[13,1,1] + X[14,1,1] + X[15,1,1] + X[16,1,1] + X[17,1,1] + X[18,1,1] + X[19,1,1] + X[20,1,1] + X[21,1,1] + X[22,1,1] + X[23,1,1] + X[24,1,1] + X[25,1,1] + X[26,1,1] + X[27,1,1] + X[28,1,1] + X[29,1,1] + X[30,1,1] + [[...40 terms omitted...]] + X[21,1,2] + X[22,1,2] + X[23,1,2] + X[24,1,2] + X[25,1,2] + X[26,1,2] + X[27,1,2] + X[28,1,2] + X[29,1,2] + X[30,1,2] + X[31,1,2] + X[32,1,2] + X[33,1,2] + X[34,1,2] + X[35,1,2] + X[36,1,2

In [18]:
@constraint(model,state_vaccination_capacity[i in n_states, j in n_weeks], sum(X[i,j,k] for k in n_doses) <= gamma * P[i])  

2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, 1:50
    Dimension 2, 1:52
And data, a 50×52 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 state_vaccination_capacity[1,1] : X[1,1,1] + X[1,1,2] ≤ 490318.5               …  state_vaccination_capacity[1,52] : X[1,52,1] + X[1,52,2] ≤ 490318.5
 state_vaccination_capacity[2,1] : X[2,1,1] + X[2,1,2] ≤ 73154.5                   state_vaccination_capacity[2,52] : X[2,52,1] + X[2,52,2] ≤ 73154.5
 state_vaccination_capacity[3,1] : X[3,1,1] + X[3,1,2] ≤ 727871.7000000001         state_vaccination_capacity[3,52] : X[3,52,1] + X[3,52,2] ≤ 727871.7000000001
 state_vaccination_capacity[4,1] : X[4,1,1] + X[4,1,2] ≤ 301780.4                  state_vaccination_capacity[4

In [19]:
# Cannot give a second shot if first shot not administered yet
second_dose_weeks = wait_period+1 : total_weeks 
@constraint(model,second_dose_constraint[i in n_states, j in second_dose_weeks], X[i,j,2] <= W[i,j-5,1] * P[i] - sum(X[i,l,2] for l in 1:j-1 )) 

2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, 1:50
    Dimension 2, 6:52
And data, a 50×47 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 second_dose_constraint[1,6] : X[1,1,2] + X[1,2,2] + X[1,3,2] + X[1,4,2] + X[1,5,2] + X[1,6,2] - 4903185 W[1,1,1] ≤ 0           …  second_dose_constraint[1,52] : X[1,1,2] + X[1,2,2] + X[1,3,2] + X[1,4,2] + X[1,5,2] + X[1,6,2] + X[1,7,2] + X[1,8,2] + X[1,9,2] + X[1,10,2] + X[1,11,2] + X[1,12,2] + X[1,13,2] + X[1,14,2] + X[1,15,2] + X[1,16,2] + X[1,17,2] + X[1,18,2] + X[1,19,2] + X[1,20,2] + X[1,21,2] + X[1,22,2] + X[1,23,2] + X[1,24,2] + X[1,25,2] + X[1,26,2] + X[1,27,2] + X[1,28,2] + X[1,29,2] + X[1,30,2] + X[1,31,2] + X[1,32,2] + X[1,33,2] + X[1,34,2] + X[1,35,2] +

In [20]:
optimize!(model)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-1060NG7 CPU @ 1.20GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 18202 rows, 13000 columns and 109500 nonzeros
Model fingerprint: 0x57700da3
Coefficient statistics:
  Matrix range     [3e-08, 4e+07]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+07]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.02s
Presolved: 9401 rows, 9600 columns, 79900 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.01s

Barrier statistics:
 AA' NZ     : 2.346e+05
 Factor NZ  : 4.322e+05 (roughly 11 MB of memory)
 Factor Ops : 2.508e+07 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Prim

In [21]:
objective_value(model)


1.4533039689024827e6

In [24]:
W = value.(W)
X = value.(X)
Y = value.(Y)

X_soln = zeros(total_states, total_weeks,2)
Y_soln = zeros(total_states, total_weeks)
W_soln = zeros(total_states, total_weeks,2)
# Compute distances between all pairs of points
for i in n_states
    for j in n_weeks
        for k in n_doses 
            X_soln[i,j,k] = X[i,j,k]
            W_soln[i,j,k] = W[i,j,k]
        Y_soln[i,j] = Y[i,j]
        end 
    end
end

# save the solutions as a jld file
save("solution_real_double_dose.jld", "X_soln", X_soln, "Y_soln", Y_soln, "W_soln", W_soln)

In [94]:
print(Y)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:50
    Dimension 2, 1:52
And data, a 50×52 Matrix{Float64}:
  2892.9                2225.1                 2736.5                2681.1000000000004    1865.9                1787.7440000000001    1341.816               920.1360000000001     601.6080000000001     586.9920000000001    586.2360000000001     357.504              677.46                269.136               225.45600000000002   175.98               263.6386629205298     253.98255835584376    176.87774174541613    132.05395644015033    109.0138240486538    362.2525449771923     76.79341852612104    69.74015304745771    88.79981987463225     58.48662835116339     43.230969872032034    62.726512655753154    76.55566800436837     117.07607192141393     173.19551249483663    319.38755059578466    367.05999999999995    478.72                521.98               396.03999999999996    775.3400000000001     554.6799999999998     514.7800000000002     413.1